<center>
    <p style="text-align:center">
        <img alt="arize llama-index logos" src="https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/arize_llamaindex.png" width="400">
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating and Improving a LlamaIndex Search and Retrieval Application</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LlamaIndex. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Download an pre-indexed knowledge base of the Arize documentation and run a LlamaIndex application
- Visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using cosine similarity, Euclidean distance, and LLM-assisted ranking metrics

Parts of this notebook require an [OpenAI API key](https://platform.openai.com/account/api-keys) to run. If you don't have an OpenAI key, you can still run Phoenix by skipping cells preceded by the 💭 emoji.


## Chatbot Architecture

Your chatbot was built using LlamaIndex's low-level API. The architecture of your chatbot is shown below and can be explained in five steps.

![llama-index chatbot architecture](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/llama_index_chatbot_architecture.png)

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to the OpenAI embeddings API to embed the user query using the `text-embedding-ada-002` model.
1. `llama_index.retrievers.RetrieverQueryEngine` does a similarity search against the entries of your index knowledge base for the two most similar pieces of context by cosine similarity.
1. `llama_index.indices.query.ResponseSynthesizer` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI chat completions API with the `gpt-3.5-turbo`.
2. The response is returned to the user.

Phoenix makes your search and retrieval system *observable* by capturing the inputs and outputs of these steps for analysis, including:

- your query embeddings
- the retrieved context and similarity scores for each query
- the generated response that is return to the user

With that overview in mind, let's dive into the notebook.

## 1. Install Dependencies and Import Libraries

Install Phoenix and LlamaIndex.

In [ ]:
!pip install -q "arize-phoenix[experimental]" gcsfs llama-index

Import libraries.

In [3]:
import os
import textwrap
from datetime import timedelta

import numpy as np
import openai
import pandas as pd
import phoenix as px
from gcsfs import GCSFileSystem
from IPython.display import YouTubeVideo
from llama_index.llms import OpenAI
from llama_index import ServiceContext, StorageContext, load_index_from_storage
from llama_index.callbacks import CallbackManager, OpenInferenceCallbackHandler
from llama_index.callbacks.open_inference_callback import as_dataframe
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from phoenix.experimental.evals.retrievals import (
    classify_relevance,
    compute_precisions_at_k,
)

pd.set_option("display.max_colwidth", 1000)


## 2. Configure Your OpenAI API Key

💭 Configure your OpenAI API key.

In [ ]:
openai_api_key = ""
assert openai_api_key != "", "❌ Please set your OpenAI API key"
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key


## 3. Download Your Knowledge Base

Download your pre-built index from cloud storage and instantiate your storage context.

In [4]:
# file_system = GCSFileSystem(project="public-assets-275721")
# index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"
index_path = "/Users/xandersong/Desktop/original-index"
storage_context = StorageContext.from_defaults(
    # fs=file_system,
    persist_dir=index_path,
)


Download and unzip a pre-built knowledge base index consisting of chunks of the Arize documentation.

## 4. Run Your Question-Answering Service

💭 Start a LlamaIndex application from your downloaded index. Use the `OpenInferenceCallbackHandler` to store your data in [OpenInference format](https://github.com/Arize-ai/open-inference-spec), an open standard for capturing and storing AI model inferences that enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Arize and Phoenix.

In [5]:
callback_handler = OpenInferenceCallbackHandler()
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()


💭 Ask a few questions of your question-answering service and view the responses.

In [6]:
max_line_length = 80
for query in [
    "How do I get an Arize API key?",
    "Can I create monitors with an API?",
    "How do I need to format timestamps?",
    "What is the price of the Arize platform",
]:
    print("Query")
    print("=====")
    print()
    print(textwrap.fill(query, max_line_length))
    print()
    response = query_engine.query(query)
    print("Response")
    print("========")
    print()
    print(textwrap.fill(str(response), max_line_length))
    print()


Query
=====

How do I get an Arize API key?

Response

To get an Arize API key, you need to click the 'Get Your API Key' button on the
top right of the Explorer page. This will generate your API key, which you can
then use to make API queries with other tools such as cURL, Postman, and client
libraries for various programming languages. Remember to keep your API key
secure and avoid sharing it in public forums or platforms.

Query
=====

Can I create monitors with an API?

Response

Yes, you can create monitors with an API.

Query
=====

How do I need to format timestamps?

Response

You can format your timestamps in either seconds or using the RFC3339 timestamp
format. If you don't provide a timestamp, it will default to the time of file
ingestion.

Query
=====

What is the price of the Arize platform

Response

I'm sorry, but I don't have access to the pricing information for the Arize
platform. For detailed pricing information, I recommend reaching out to Arize's
sales team at conta

## 5. Load Your Data Into Pandas Dataframes

To use Phoenix, you must load your data into pandas dataframes. 

💭 Your query data is saved in a buffer on the callback handler you defined in step 4. Load the data from the buffer into a dataframe.

In [7]:
query_data_buffer = callback_handler.flush_query_data_buffer()
sample_query_df = as_dataframe(query_data_buffer)
sample_query_df


,:id.id:,:timestamp.iso_8601:,:feature.text:prompt,:feature.[float].embedding:prompt,:prediction.text:response,:feature.[str].retrieved_document_ids:prompt,:feature.[float].retrieved_document_scores:prompt
0,93dddd06-a82a-4cc1-800f-d69374d18065,2023-08-18T18:38:08.199810,How do I get an Arize API key?,"[0.007174017373472452, -0.01062347088009119, -0.022770626470446587, -0.02487274818122387, -0.014870037324726582, 0.025916755199432373, 0.005650332197546959, 0.0013605590211227536, -0.014122303575277328, -0.033097825944423676, 0.019074279814958572, -0.007427964825183153, -0.0008870527381077409, 0.0028040038887411356, -0.025648698210716248, 0.015674205496907234, 0.016041018068790436, 0.0073715317994356155, 0.03402896597981453, -0.00714580062776804, -0.020837804302573204, 0.0011806795373558998, 0.015067552216351032, -0.011046716943383217, -0.014404467307031155, 0.013289920054376125, -0.0021850066259503365, 0.004482878837734461, 0.003646968398243189, -0.028075307607650757, 0.03123554401099682, 0.02224862389266491, -0.0008337061735801399, -0.0031990332063287497, -0.033069610595703125, 0.004980192985385656, 0.019723257049918175, 0.010073251090943813, 0.006073577795177698, -0.0009655295871198177, 0.02116229198873043, -0.018213680014014244, 0.013155891560018063, -0.012196534313261509, -0.0...","To get an Arize API key, you need to click the 'Get Your API Key' button on the top right of the Explorer page. This will generate your API key, which you can then use to make API queries with other tools such as cURL, Postman, and client libraries for various programming languages. Remember to keep your API key secure and avoid sharing it in public forums or platforms.","[a1f87cc6-05e1-4491-ba17-93b5d0a956ed, d060a50c-4c4e-42be-80f7-76c381289567]","[0.9057707464041789, 0.8818316306728519]"
1,54bc1e49-befc-4d66-a762-f7903745a5e9,2023-08-18T18:38:10.574175,Can I create monitors with an API?,"[-0.006972833536565304, -0.02340657263994217, 0.0005138787673786283, -0.02674246020615101, -0.0033012819476425648, 0.009433222934603691, -0.022354591637849808, -0.004007216542959213, -0.002553821774199605, -0.012921369634568691, 0.011682523414492607, 0.005817038472741842, 0.005426006391644478, 0.008069800212979317, -0.023115893825888634, 0.016139600425958633, 0.022050071507692337, 0.01631954498589039, 0.014852307736873627, -0.022050071507692337, -0.012436904944479465, 0.009495510719716549, 0.005045355297625065, 0.0036507886834442616, -0.026894720271229744, 0.010852012783288956, 0.0001205755106639117, -0.015516716986894608, -0.029427777975797653, -0.002519217086955905, 0.043850988149642944, -0.005041894968599081, -0.014658521860837936, -0.022562220692634583, -0.014866149984300137, 0.0007085298420861363, 0.02145487256348133, 0.024970702826976776, 0.013426597230136395, 0.004038360435515642, 0.022506851702928543, 0.015405981801450253, 0.016831692308187485, -0.006055810488760471, 0.0011...","Yes, you can create monitors with an API.","[b6d0b8fc-7891-4f45-bf77-6028cb453ce6, 6a710d0f-2935-4565-887a-eb4c33d1cd6a]","[0.8482175188606403, 0.8406339812286953]"
2,019dbfc6-a012-4d7c-b692-cdb8a643ada0,2023-08-18T18:38:11.261316,How do I need to format timestamps?,"[-0.015752878040075302, 0.008697481825947762, -0.005625531077384949, -0.04291685298085213, -0.004533127415925264, -0.009727262891829014, -0.0034668161533772945, -0.015029247850179672, -0.01593378558754921, -0.008634859696030617, 0.01533539965748787, 0.016239937394857407, -0.0141734154894948, -0.005642925854772329, -0.018243836238980293, 0.0020613030064851046, 0.010005583055317402, 0.013561112806200981, 0.02428336814045906, -0.021945485845208168, -0.026148108765482903, 0.014764844439923763, 0.00973422173410654, -0.00695102708414197, 0.007340674288570881, -0.0010332608362659812, -0.001777765224687755, -0.006276102736592293, -0.0016838323790580034, -0.003858202835544944, 0.013769852928817272, -0.010889247059822083, -0.001821252633817494, -0.029112210497260094, -0.017895936965942383, -0.011299767531454563, -0.010513515

In [ ]:
query_data_buffer[0].response_text


The columns of the dataframe are:

- **:id.id:**: the query ID
- **:timestamp.iso_8601:**: the time at which the query was made
- **:feature.text:prompt**: the query text
- **:feature.[float].embedding:prompt**: the embedding representation of the query
- **:prediction.text:response**: the final response presented to the user
- **:feature.[str].retrieved_document_ids:prompt**: the list of IDs of the retrieved documents
- **:feature.[float].retrieved_document_scores:prompt**: the lists of cosine similarities between the query and retrieved documents

The column names are in OpenInference format and describe the category, data type and intent of each column.

Running queries against a large dataset takes a long time. Download a dataframe containing query data.

In [8]:
query_df = pd.read_parquet(
    # "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/query_data_complete3.parquet",
    "/Users/xandersong/Desktop/query_data.parquet"
)
query_df.head()


,:id.id:,:timestamp.iso_8601:,:feature.text:prompt,:feature.[float].embedding:prompt,:prediction.text:response,:feature.[str].retrieved_document_ids:prompt,:feature.[float].retrieved_document_scores:prompt,:tag.bool:relevance_0,:tag.bool:relevance_1,:tag.float:precision_at_1,:tag.float:precision_at_2,:tag.float:document_similarity_0,:tag.float:document_similarity_1,:tag.float:user_feedback
0,f1102260-5c40-4026-897e-7b5029bb6b06,2023-08-18T18:25:29.713008,How do I use the SDK to upload a ranking model?,"[-0.009536871686577797, 0.008512362837791443, 0.013284236192703247, -0.0024701666552573442, 0.009846286848187447, 0.02992391213774681, -0.019211260601878166, -0.019087495282292366, 0.005610733292996883, -0.03492956608533859, 0.02467072568833828, -0.013057331554591656, -0.016777193173766136, -0.01948629692196846, -0.022594204172492027, -0.00867738388478756, 0.013456134125590324, 0.003432792378589511, 0.008017297834157944, 0.0022381050512194633, -0.016515908762812614, 0.017973599955439568, 0.000521708803717047, -0.014439387246966362, -0.011668399907648563, 0.015250743366777897, 0.021700337529182434, -0.01339425053447485, 0.01630963198840618, 0.002007762435823679, 0.001018492504954338, 0.011455247178673744, -0.021095257252454758, -0.008017297834157944, -0.018331145867705345, 0.012892310507595539, -0.009103690274059772, 0.008443604223430157, 0.01030009612441063, -0.02304801344871521, 0.020943988114595413, 0.018523670732975006, 0.0022191964089870453, -0.022319167852401733, -0.0044762101...","You can use the Arize SDK to upload a ranking model by following the instructions in the documentation. The documentation can be found at https://arize.com/docs/sdk/upload-ranking-model.html. If you have any questions or need help, you can contact us at support@arize.com.","[fa56d2c6-1dd8-488b-86f5-201e8df72754, 1ec49012-1319-4d12-bab1-2693b9e0d062]","[0.8414428747761786, 0.8242186029984772]",False,False,0.0,0.0,0.841443,0.824219,-1.0
1,144ad1bf-ebb7-4d83-9a32-0e15fe38b11d,2023-08-18T18:25:32.775250,What drift metrics are supported in Arize?,"[-0.009346794337034225, -0.00042854511411860585, -0.0009247552370652556, -0.017379986122250557, -0.0047960965894162655, 0.030313927680253983, -0.016629355028271675, 0.006423665676265955, -0.027657851576805115, -0.017495466396212578, 0.01267410907894373, 0.019270997494459152, -0.01836157962679863, -0.005113671068102121, -0.01797182857990265, 0.005756037775427103, 0.0026596863754093647, 0.003597974544391036, 0.02191263996064663, -0.006073612254112959, -0.015792112797498703, -0.017293374985456467, -0.003929984290152788, 0.008646687492728233, -0.011115106754004955, 0.0181161817163229, 0.01463007926940918, -0.005117279943078756, 0.015128093771636486, -0.02220134437084198, 0.04408511146903038, 0.027499062940478325, -0.00033742288360372186, 0.014211458154022694, -0.034817710518836975, -0.011475986801087856, 0.007636222988367081, 0.020497988909482956, 0.028119778260588646, -0.011779126711189747, 0.0045903949066996574, -0.017235632985830307, 0.0013929972192272544, -0.005752428900450468, 0.0...","Arize supports drift metrics such as Population Stability Index, KL Divergence, and Wasserstein Distance.","[27772b91-5978-4930-affb-510636ef8032, 807b8181-9dac-46ef-aae7-6aa81df99510]","[0.8988759026583779, 0.8812275254570013]",True,True,1.0,1.0,0.898876,0.881228,1.0
2,29af7a8d-773b-4ed2-b3ee-6bc42ef1cdad,2023-08-18T18:25:34.003974,Does Arize support batch models?,"[-0.016511010006070137, -0.000836528604850173, -0.016215920448303223, -0.007496701553463936, 0.0061793336644768715, 0.018225345760583878, 0.026698654517531395, 0.0029333392158150673, -0.008684088476002216, -0.035551365464925766, 0.007904207333922386, 0.025855539366602898, 0.0010196426883339882, -0.008346842601895332, -0.01403084583580494, 0.004577414132654667, 0.012492160312831402, 0.017831891775131226, 0.023171622306108475, -0.006569274235516787, -0.013180704787373543, -0.008740296587347984, 0.008466283790767193, -0.019827265292406082, 0.008951075375080109, 0.031

In addition to the columns of the previous dataframe, this data has a few additional fields:

- **:tag.float:user_feedback:** approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)
- **:tag.str:openai_relevance_0:** a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the first retrieved document is relevant to the query
- **:tag.str:openai_relevance_1:** a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the second retrieved document is relevant to the query

We'll go over how to compute the relevance classifications in section 6.

Next load your knowledge base into a dataframe.

In [12]:
def storage_context_to_dataframe(storage_context: StorageContext) -> pd.DataFrame:
    """Converts the storage context to a pandas dataframe.

    Args:
        storage_context (StorageContext): Storage context containing the index
        data.

    Returns:
        pd.DataFrame: The dataframe containing the index data.
    """
    document_ids = []
    document_texts = []
    document_embeddings = []
    docstore = storage_context.docstore
    vector_store = storage_context.vector_store
    for node_id, node in docstore.docs.items():
        document_ids.append(node.node_id)  # use node hash as the document ID
        document_texts.append(node.text)
        document_embeddings.append(np.array(vector_store.get(node_id)))
    return pd.DataFrame(
        {
            "document_id": document_ids,
            "text": document_texts,
            "text_vector": document_embeddings,
        }
    )


database_df = storage_context_to_dataframe(storage_context)
database_df = database_df.drop_duplicates(subset=["text"])
database_df.head()


,document_id,text,text_vector
0,255a418f-c1a9-4444-8451-68e2a64a1106,Tutorials\n\nAccess tutorials of what's possible with Arize below:\n\n1. Model Type Examples\n2. Explainability Tutorials\n3. Cloud Storage Examples,"[-0.002403473947197199, 0.004425443708896637, 0.015301769599318504, -0.011056673713028431, 0.0060034822672605515, 0.012991937808692455, 0.014511017128825188, -0.015482117421925068, -0.010154937393963337, -0.052050989121198654, -0.0013248586328700185, -0.006149147171527147, -0.011542223393917084, -0.0018000042764469981, -0.008587303571403027, 0.004258969333022833, 0.008413892239332199, 0.007262444589287043, 0.01746593788266182, -0.024499479681253433, -0.020171144977211952, -0.00798383355140686, -0.014913329854607582, -0.01627287082374096, 0.001917923567816615, 0.018880968913435936, 0.02486017346382141, -0.01684165745973587, 0.0078104231506586075, -0.017452064901590347, 0.024901792407035828, 0.017923740670084953, 0.004831225145608187, -0.038150377571582794, -0.006738744210451841, 0.006849727127701044, 0.02279311791062355, 0.023930691182613373, -0.002854341873899102, -0.016550328582525253, 0.03498736396431923, -0.02040698379278183, -0.0006823715521022677, -0.0029913363978266716, -0.00..."
1,d7c4a61f-6ba0-44a3-8d72-026bc6ed100c,"Model Type Examples\n\n{% hint style=""info"" %}\nYour model type determines which performance metrics are available to you. Learn more about model types here.\n{% endhint %}\n\nModel TypePandas BatchPython Single RecordCSVParquet Binary Classification (Only Classification Metrics)Colab LinkColab LinkDownload File*Open Parquet Reader HereBinary Classification (Classification, AUC/Log Loss Metrics) Colab LinkColab LinkDownload File*Open Parquet Reader HereBinary Classification (Classification, AUC/Log Loss, Regression) Colab LinkColab LinkDownload FileDownload File*Open Parquet Reader HereMulticlass Classification (Only Classification Metrics)Colab LinkColab LinkDownload File*Open Parquet Reader HereMulticlass Classification (Classification, AUC/Log Loss Metrics)Colab LinkColab LinkDownload File*Open Parquet Reader HereRegressionColab LinkColab LinkDownload FileDownload File*Open Parquet Reader HereTimeseries Forecasting Colab LinkColab LinkDownload File*Open Parquet Reader HereRankin...","[-0.0005261292681097984, 0.0032938774675130844, 0.02757122926414013, -0.030902810394763947, -0.00580284558236599, 0.015328013338148594, 0.0008046178263612092, -0.01093389093875885, -0.020016904920339584, -0.03712724521756172, 0.0022998873610049486, 0.03605784848332405, -0.02665264531970024, -0.0025689504109323025, -0.031862523406744, -0.0057308669202029705, 0.01948220655322075, -0.002515823347494006, 0.030107617378234863, -0.00027206193772144616, -0.015286882407963276, 0.015807870775461197, -0.02274523489177227, -0.009000752121210098, -0.02556953951716423, 0.01768616959452629, 0.0358659066259861, -0.020469341427087784, -0.0014378580963239074, -0.001212496543303132, 0.013477134518325329, 0.010371772572398186, -0.023979155346751213, -0.03614011034369469, -0.006310123484581709, -1.692353907856159e-05, 0.02403399534523487, 0.01815231516957283, 0.004065076820552349, -0.0021867782343178988, 0.0030847969464957714, 0.012551696039736271, 0.006268993020057678, -0.00972739327698946, 0.0239517..."
2,a34cf3de-3de9-407d-88bc-9bf5b8f34642,Explainability Tutorials\n\nExamples for logging explainability metrics. Click here for more information on how to log feature importance and use explainability.\n\n| SHAP: Guide to Getting Started | Colab Link |\n| ------------------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |\n| SHAP: Neural Network on Tabular Data | Colab Link |\n| Surrogate Model Explainability | Colab Link |\n| One Hot Encoding Decomposition | Colab Link |,"[-8.327089744852856e-06, -0.002697667805477977, 0.04392826184630394, -0.027323167771100998, 0.00855333916842937

The columns of your dataframe are:

- **document_id:** the ID of the chunked document
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text, computed during the LlamaIndex build using "text-embedding-ada-002" from OpenAI

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from OpenAI's "text-embedding-ada-002" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [13]:
database_centroid = database_df["text_vector"].mean()
database_df["text_vector"] = database_df["text_vector"].apply(lambda x: x - database_centroid)

query_centroid = query_df[":feature.[float].embedding:prompt"].mean()
query_df[":feature.[float].embedding:prompt"] = query_df[":feature.[float].embedding:prompt"].apply(
    lambda x: x - query_centroid
)


## 6. Run LLM-Assisted Evaluations

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each retrieved document is relevant to the corresponding query.

💭 Use OpenAI to predict whether each retrieved document is relevant or irrelevant to the query.

⚠️ It's strongly recommended to use GPT-4 for evaluations if you have access.

In [15]:
evals_model_name = "gpt-3.5-turbo"
# evals_model_name = "gpt-4"  # use GPT-4 if you have access
query_texts = sample_query_df[":feature.text:prompt"].tolist()
list_of_document_id_lists = sample_query_df[":feature.[str].retrieved_document_ids:prompt"].tolist()
document_id_to_text = dict(zip(database_df["document_id"].to_list(), database_df["text"].to_list()))

first_document_texts, second_document_texts = [
    [
        document_id_to_text[document_ids[document_index]]
        for document_ids in list_of_document_id_lists
    ]
    for document_index in [0, 1]
]
first_document_relevances, second_document_relevances = [
    [
        classify_relevance(query_text, document_text, evals_model_name)
        for query_text, document_text in zip(query_texts, first_document_texts)
    ]
    for document_texts in [first_document_texts, second_document_texts]
]


sample_query_df = sample_query_df.assign(
    retrieved_document_text_0=first_document_texts,
    retrieved_document_text_1=second_document_texts,
    relevance_0=first_document_relevances,
    relevance_1=second_document_relevances,
)
sample_query_df[
    [
        ":feature.text:prompt",
        "retrieved_document_text_0",
        "retrieved_document_text_1",
        "relevance_0",
        "relevance_1",
    ]
].rename(columns={":feature.text:prompt": "query_text"})


,query_text,retrieved_document_text_0,retrieved_document_text_1,relevance_0,relevance_1
0,How do I get an Arize API key?,"Access the API Key&x20;\n\nUse the API key to make API queries with other tools such as cURL, Postman, and client libraries for various programming languages.&#x20;\n\nClick the 'Get Your API Key' button on the top right of the Explorer page to generate your API key.&#x20;\n\n!Access API Key\n\n{% hint style=""info"" %}\nHaving trouble? Reach out to us via email support@arize.com or Slack us in the #arize-support channel for more support.&#x20;\n{% endhint %}","Authentication\n\nArize uses API keys in conjunction with a Space key in order to authenticate request. You will receive your keys once you sign up for our product.\\n\\nYour API and Space keys are the keys to your data castle so be sure to keep them secure. Do not share your keys in public forums including Github, StackOverflow, logs, etc.\\n\\nThe API key is authenticated via auth header, use `-H 'Authorization: API_KEY_VALUE'` while the Space key is included on the body of the request",True,True
1,Can I create monitors with an API?,How to Setup Monitors\n\nYou can configure monitors in **three ways**:\n\nMethodManaged MonitorsEnable monitors in a few clicks for a simple setup #managed-monitorsCustom MonitorsCustomize monitors for the dimensions that matter the most to you#custom-monitorsProgrammaticallyUse our public-facing graphQL API to configure monitors#programmatically,How To Edit: Custom Monitors,True,True
2,How do I need to format timestamps?,"How should I format my prediction timestamp?\n\nYour timestamp can be in seconds or an RFC3339 timestamp. If you do not specify a timestamp, your timestamp will default to the time of file ingestion.&#x20;",Timestamps Q\&A,True,True
3,What is the price of the Arize platform,"Platform and Model Agnostic&x20;\n\nArize is an open platform that works with your machine learning infrastructure, and can be deployed as SaaS or on-premise.\n\n!Open Platform designed to work across platforms and model frameworks","Overview\n\nWe also offer Arize On-Prem, which runs a completely scalable infrastructure within your company's AWS, GCP or Azure account. The environment can be provisioned by us or by your company, using a toolset comprised of Terraform and Kubernetes.\n\nTalk to our sales team for more detailed setup instructions by reaching out to contacts@arize.com",False,False


## 7. Compute Ranking Metrics

Now that you know whether each piece of retrieved context is relevant or irrelevant to the corresponding query, you can compute precision@k for k = 1, 2 for each query. This metric tells you what percentage of the retrieved context is relevant to the corresponding query.

precision@k = (# of top-k retrieved documents that are relevant) / (k retrieved documents)

If your precision@2 is greater than zero for a particular query, your LlamaIndex application successfully retrieved at least one relevant piece of context with which to answer the query. If the precision@k is zero for a particular query, that means that no relevant piece of context was retrieved.

💭 Compute precision@k for k = 1, 2 and view the results.

In [21]:
list_of_precisions_at_k_lists = [
    compute_precisions_at_k([rel0, rel1])
    for rel0, rel1 in zip(
        sample_query_df["relevance_0"].tolist(), sample_query_df["relevance_1"].tolist()
    )
]
precisions_at_1, precisions_at_2 = [
    [precisions_at_k[index] for precisions_at_k in list_of_precisions_at_k_lists]
    for index in [0, 1]
]
sample_query_df["precision_at_1"] = precisions_at_1
sample_query_df["precision_at_2"] = precisions_at_2
sample_query_df[
    [
        "relevance_0",
        "relevance_1",
        "precision_at_1",
        "precision_at_2",
    ]
]


,relevance_0,relevance_1,precision_at_1,precision_at_2
0,True,True,1.0,1.0
1,True,True,1.0,1.0
2,True,True,1.0,1.0
3,False,False,0.0,0.0


## 7. Launch Phoenix

Define your knowledge base dataset with a schema that specifies the meaning of each column (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [22]:
# get a random sample of 500 documents (including retrieved documents)
# this will be handled by by the application in a coming release
num_sampled_point = 500
retrieved_document_ids = set(
    [
        doc_id
        for doc_ids in query_df[":feature.[str].retrieved_document_ids:prompt"].to_list()
        for doc_id in doc_ids
    ]
)
retrieved_document_mask = database_df["document_id"].isin(retrieved_document_ids)
num_retrieved_documents = len(retrieved_document_ids)
num_additional_samples = num_sampled_point - num_retrieved_documents
unretrieved_document_mask = ~retrieved_document_mask
sampled_unretrieved_document_ids = set(
    database_df[unretrieved_document_mask]["document_id"]
    .sample(n=num_additional_samples, random_state=0)
    .to_list()
)
sampled_unretrieved_document_mask = database_df["document_id"].isin(
    sampled_unretrieved_document_ids
)
sampled_document_mask = retrieved_document_mask | sampled_unretrieved_document_mask
sampled_database_df = database_df[sampled_document_mask]


In [23]:
database_schema = px.Schema(
    prediction_id_column_name="document_id",
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="text_vector",
        raw_data_column_name="text",
    ),
)
database_ds = px.Dataset(
    dataframe=sampled_database_df,
    schema=database_schema,
    name="database",
)


Define your query dataset. Because the query dataframe is in OpenInference format, Phoenix is able to infer the meaning of each column without a user-defined schema by using the `phoenix.Dataset.from_open_inference` class method.

In [24]:
query_ds = px.Dataset.from_open_inference(query_df)


Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [25]:
session = px.launch_app(primary=query_ds, corpus=database_ds)


None
🌍 To view the Phoenix app in your browser, visit http://localhost:54290/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## 8. Surface Problematic Clusters and Data Points

Phoenix helps you:

- visualize your embeddings
- color the resulting point cloud using evaluation metrics
- cluster the points and surface up problematic clusters based on whatever metric you care about

Follow along with the tutorial walkthrough [here](https://youtu.be/hbQYDpJayFw?t=1782), or view the video in your notebook by running the cell below. The video will show you how to investigate your query and knowledge base and identify problematic clusters of data points using Phoenix.

In [ ]:
start_time_in_seconds = int(timedelta(hours=0, minutes=29, seconds=42).total_seconds())
YouTubeVideo("hbQYDpJayFw", start=start_time_in_seconds, width=560, height=315)


Congrats! You've identified a problematic cluster of queries. You now have tools at your disposal to investigate clusters of queries where your search and retrieval application is performing poorly based on:

- query percentage
- user feedback
- LLM-assisted ranking metrics

As an actionable next step, you should augment your knowledge base to include information about the pricing and cost of the Arize platform, since your users seem especially interested in this topic.